# Agentic Design Patterns: Reflection

![alt text](image.png)

## Setup

In [1]:
from utils import get_openai_api_key
OPENAI_API_KEY = get_openai_api_key()

In [2]:
llm_config = {"model": "gpt-3.5-turbo"}

## The task!

In [3]:
task = '''
        Write a concise but engaging blogpost about
        Data Saturday Dallas. Make sure the blogpost is
       within 100 words.
       '''


## Create a writer agent

In [4]:
import autogen

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

[autogen.oai.client: 09-07 10:35:32] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [5]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

In [6]:
print(reply)

Title: "Unveiling the Data Wonders of Data Saturday Dallas!"

Get ready to dive into a world of data excellence at Data Saturday Dallas! This event is a gold mine of insights, networking opportunities, and hands-on learning for data enthusiasts. With top-notch speakers and cutting-edge topics, you'll be empowered to unlock the true potential of data. Whether you're a seasoned data professional or just starting your journey, Data Saturday Dallas has something for everyone. Don't miss this chance to level up your data skills and connect with like-minded individuals. Mark your calendars for an unforgettable experience! 🚀📊 #DataSaturdayDallas #DataExcellence


## Adding reflection 

Create a critic agent to reflect on the work of the writer agent.

In [7]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

[autogen.oai.client: 09-07 10:36:14] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [8]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=3,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
        Data Saturday Dallas. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: "Unveiling the Data Wonders of Data Saturday Dallas!"

Get ready to dive into a world of data excellence at Data Saturday Dallas! This event is a gold mine of insights, networking opportunities, and hands-on learning for data enthusiasts. With top-notch speakers and cutting-edge topics, you'll be empowered to unlock the true potential of data. Whether you're a seasoned data professional or just starting your journey, Data Saturday Dallas has something for everyone. Don't miss this chance to level up your data skills and connect with like-minded individuals. Mark your calendars for an unforgettable experience! 🚀📊 #DataSaturdayDallas #DataExcellence

----------------------------------------------------------------------

## Nested chat

In [9]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)


[autogen.oai.client: 09-07 10:40:05] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [10]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

[autogen.oai.client: 09-07 10:40:11] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [11]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

[autogen.oai.client: 09-07 10:40:19] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [12]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

[autogen.oai.client: 09-07 10:40:28] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


## Orchestrate the nested chats to solve the task

In [13]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]


In [14]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different task.

In [15]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
        Data Saturday Dallas. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: "Unveiling the Data Wonders of Data Saturday Dallas!"

Get ready to dive into a world of data excellence at Data Saturday Dallas! This event is a gold mine of insights, networking opportunities, and hands-on learning for data enthusiasts. With top-notch speakers and cutting-edge topics, you'll be empowered to unlock the true potential of data. Whether you're a seasoned data professional or just starting your journey, Data Saturday Dallas has something for everyone. Don't miss this chance to level up your data skills and connect with like-minded individuals. Mark your calendars for an unforgettable experience! 🚀📊 #DataSaturdayDallas #DataExcellence

----------------------------------------------------------------------

## Get the summary

In [16]:
print(res.summary)

Title: "Unlock the Data Magic at Data Saturday Dallas!"

Embark on a data journey at Data Saturday Dallas, a hub of knowledge and networking for data professionals. Immerse yourself in top-tier speakers and cutting-edge topics to elevate your data skills. Prioritizing data privacy, we collect information with explicit consent for transparent networking benefits. Upholding compliance, we substantiate our event value to provide accurate promises. Optimize your experience by using specific hashtags and keywords for seamless search visibility. Join us for an enriching, transparent, and compliant event that respects your data and empowers your growth! 🌟📊 #DataSaturdayDallas #DataMagic
